In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/Data/Cleaned_Null_Values+Sale_Data.csv')
df

,Unnamed: 0,status,price,bed,bath,acre_lot,city,state,zip_code,house_size,months_since_prev_sold
0,0,for_sale,105000.0,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.000000,0
1,1,for_sale,80000.0,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.000000,0
2,2,for_sale,67000.0,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.000000,0
3,3,for_sale,145000.0,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.000000,0
4,4,for_sale,65000.0,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,2714.471335,0
...,...,...,...,...,...,...,...,...,...,...,...
1232461,1235031,for_sale,120000.0,3.0,2.0,4.00,Rio Rancho,New Mexico,87144.0,2714.471335,0
1232462,1235032,for_sale,16000.0,3.0,2.0,0.50,Rio Rancho,New Mexico,87144.0,2714.471335,0
1232463,1235033,for_sale,30000.0,3.0,2.0,1.00,Rio Rancho,New Mexico,87144.0,2714.471335,0
1232464,1235034,for_sale,30000.0,3.0,2.0,1.00,Rio Rancho,New Mexico,87144.0,2714.471335,0


## Feature Engineering

In [3]:
"""
Cell generated by Data Wrangler.
"""
import pickle
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def clean_data(df):
    os.makedirs('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/pipelines',exist_ok=True)
    le = LabelEncoder()    
    one = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    status_encoded = one.fit_transform(df[['status']])
    with open('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/pipelines/one_hot_encoding.pkl','wb') as f:
        pickle.dump(one, f)  # save encoder, not data
    status_encoded_df = pd.DataFrame(status_encoded,columns=one.get_feature_names_out(['status']))
    df = pd.concat([df.drop(['status'],axis=1),status_encoded_df],axis=1) 
    df['city'] = le.fit_transform(df['city'] + ", " + df['state'])
    with open('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/pipelines/label_encoding.pkl','wb') as f:
        pickle.dump(le, f)
    df = df.drop(['state'],axis=1)
    df['acre_lot'] = df['acre_lot'] * 43560
    df['total_house_size'] = df['acre_lot'] + df['house_size']
    df['total_rooms'] = df['bed'] + df['bath']
    df['price_per_sqft'] = df['total_house_size'] / df['price']
    df = df.drop(columns=df.columns[df.isin([np.inf, -np.inf]).any()])
    df = df.rename(columns={'price':'cost',
                            'bed':'bedrooms',
                            'bath':'bathrooms',
                            'acre_lot':'sqft_lot',
                            'house_size':'house_lot',
                            'city':'location',
                            'months_since_prev_sold':'last_sold(months)'})
    # Round columns 'cost', 'sqft_lot' and 3 other columns (Number of decimals: 4)
    df = df.round({'cost': 4, 'sqft_lot': 4, 'house_lot': 4, 'total_house_size': 4, 'price_per_sqft': 4})
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,Unnamed: 0,cost,bedrooms,bathrooms,sqft_lot,location,zip_code,house_lot,last_sold(months),status_for_sale,status_ready_to_build,total_house_size,total_rooms
0,0,105000.0,3.0,2.0,5227.2,129,601.0,920.0000,0,1.0,0.0,6147.2000,5.0
1,1,80000.0,4.0,2.0,3484.8,129,601.0,1527.0000,0,1.0,0.0,5011.8000,6.0
2,2,67000.0,2.0,1.0,6534.0,11999,795.0,748.0000,0,1.0,0.0,7282.0000,3.0
3,3,145000.0,4.0,2.0,4356.0,19112,731.0,1800.0000,0,1.0,0.0,6156.0000,6.0
4,4,65000.0,6.0,2.0,2178.0,14761,680.0,2714.4713,0,1.0,0.0,4892.4713,8.0


In [4]:
df = df_clean.copy()

In [5]:
df

,Unnamed: 0,cost,bedrooms,bathrooms,sqft_lot,location,zip_code,house_lot,last_sold(months),status_for_sale,status_ready_to_build,total_house_size,total_rooms
0,0,105000.0,3.0,2.0,5227.2,129,601.0,920.0000,0,1.0,0.0,6147.2000,5.0
1,1,80000.0,4.0,2.0,3484.8,129,601.0,1527.0000,0,1.0,0.0,5011.8000,6.0
2,2,67000.0,2.0,1.0,6534.0,11999,795.0,748.0000,0,1.0,0.0,7282.0000,3.0
3,3,145000.0,4.0,2.0,4356.0,19112,731.0,1800.0000,0,1.0,0.0,6156.0000,6.0
4,4,65000.0,6.0,2.0,2178.0,14761,680.0,2714.4713,0,1.0,0.0,4892.4713,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232461,1235031,120000.0,3.0,2.0,174240.0,20157,87144.0,2714.4713,0,1.0,0.0,176954.4713,5.0
1232462,1235032,16000.0,3.0,2.0,21780.0,20157,87144.0,2714.4713,0,1.0,0.0,24494.4713,5.0
1232463,1235033,30000.0,3.0,2.0,43560.0,20157,87144.0,2714.4713,0,1.0,0.0,46274.4713,5.0
1232464,1235034,30000.0,3.0,2.0,43560.0,20157,87144.0,2714.4713,0,1.0,0.0,46274.4713,5.0


In [6]:
df.isna().sum()

Unnamed: 0               0
cost                     0
bedrooms                 0
bathrooms                0
sqft_lot                 0
location                 0
zip_code                 0
house_lot                0
last_sold(months)        0
status_for_sale          0
status_ready_to_build    0
total_house_size         0
total_rooms              0
dtype: int64

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = pd.DataFrame(
    scaler.fit_transform(df.drop(['cost'], axis=1)),
    columns=scaler.feature_names_in_
)

y = df['cost']

final_df = pd.concat([X,y],axis=1)


In [18]:
final_df

,Unnamed: 0,bedrooms,bathrooms,sqft_lot,location,zip_code,house_lot,last_sold(months),status_for_sale,status_ready_to_build,total_house_size,total_rooms,cost
0,-1.731260,-0.175324,-0.271446,-0.023478,-1.743783,-1.836941,-0.002441,-0.667847,0.13727,-0.13727,-0.023545,-0.257716,105000.0
1,-1.731257,0.585274,-0.271446,-0.023538,-1.743783,-1.836941,-0.001793,-0.667847,0.13727,-0.13727,-0.023584,0.158810,80000.0
2,-1.731254,-0.935923,-0.970654,-0.023433,-0.153825,-1.828805,-0.002624,-0.667847,0.13727,-0.13727,-0.023506,-1.090768,67000.0
3,-1.731252,0.585274,-0.271446,-0.023508,0.798945,-1.831489,-0.001502,-0.667847,0.13727,-0.13727,-0.023544,0.158810,145000.0
4,-1.731249,2.106471,-0.271446,-0.023583,0.216139,-1.833628,-0.000526,-0.667847,0.13727,-0.13727,-0.023588,0.991862,65000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232461,1.732356,-0.175324,-0.271446,-0.017633,0.938920,1.792364,-0.000526,-0.667847,0.13727,-0.13727,-0.017640,-0.257716,120000.0
1232462,1.732358,-0.175324,-0.271446,-0.022906,0.938920,1.792364,-0.000526,-0.667847,0.13727,-0.13727,-0.022911,-0.257716,16000.0
1232463,1.732361,-0.175324,-0.271446,-0.022152,0.938920,1.792364,-0.000526,-0.667847,0.13727,-0.13727,-0.022158,-0.257716,30000.0
1232464,1.732364,-0.175324,-0.271446,-0.022152,0.938920,1.792364,-0.000526,-0.667847,0.13727,-0.13727,-0.022158,-0.257716,30000.0


In [19]:
final_df.to_csv('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/Data/Cleaned_Null_Values+Sale_Data+Feature_Engineering.csv')